In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/restaurant-revenue-prediction/train.csv.zip
/kaggle/input/restaurant-revenue-prediction/sampleSubmission.csv
/kaggle/input/restaurant-revenue-prediction/test.csv.zip


In [2]:
import pandas as pd
train_data = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/train.csv.zip')
test_data = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/test.csv.zip')
test = pd.read_csv('/kaggle/input/restaurant-revenue-prediction/test.csv.zip')

In [3]:
train_data.head(1)

,Id,Open Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P29,P30,P31,P32,P33,P34,P35,P36,P37,revenue
0,0,07/17/1999,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,3.0,5,3,4,5,5,4,3,4,5653753.0


In [4]:
train_data['City_grp'] = (train_data['City Group'] =='Big Cities')-0
test_data['City_grp'] = (test_data['City Group'] =='Big Cities')-0
train_data['City_grp']

0      1
1      1
2      0
3      0
4      0
      ..
132    0
133    1
134    0
135    1
136    1
Name: City_grp, Length: 137, dtype: int64

In [5]:
train_data['Type'].unique()

array(['IL', 'FC', 'DT'], dtype=object)

In [6]:
train_data['Type'].replace(['IL','FC','DT'],[0,1,2],inplace=True)
test_data['Type'].replace(['IL','FC','DT'],[0,1,2],inplace=True)


In [7]:
l=len((train_data['City'].unique()))
list =[]
for i in range(l):
    list.append(i)
l= len((test_data['City'].unique()))
list_test=[]
for i in range(l):
    list_test.append(i)

In [8]:
train_data['City'].replace(train_data['City'].unique(),list,inplace=True)
test_data['City'].replace(test_data['City'].unique(),list_test,inplace=True)

In [9]:
train_data.drop(['City Group'],axis=1,inplace=True)
test_data.drop(['City Group'],axis=1,inplace=True)


In [10]:
train_data.drop(['Id'],axis=1,inplace=True)
test_data.drop(['Id'],axis=1,inplace=True)

In [11]:
from datetime import datetime
data = []
for i in range(train_data.shape[0]):
    data.append((datetime.now()-datetime.strptime(train_data['Open Date'][i],'%m/%d/%Y')).days)
train_data['Age'] = data 
data = []
for i in range(test_data.shape[0]):
    data.append((datetime.now()-datetime.strptime(test_data['Open Date'][i],'%m/%d/%Y')).days)
test_data['Age'] = data

In [12]:
train_data['Age'] = train_data['Age'] / 1000
test_data['Age'] = test_data['Age'] / 1000
train_data.head()
train_data.drop(['Open Date'],axis=1,inplace=True)
test_data.drop(['Open Date'],axis=1,inplace = True)


In [13]:
X_train = train_data.drop(['revenue'],axis=1)
target = train_data['revenue'] / 1e7


In [14]:
import tensorflow as tf
tf.random.set_seed(42)


In [15]:
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])
model = tf.keras.Sequential([norm_layer,
                           tf.keras.layers.Dense(400,activation='relu'),
                           tf.keras.layers.Dense(300,activation='relu'),
                           tf.keras.layers.Dense(200,activation='relu'),
                           tf.keras.layers.Dense(100,activation='relu'),
                           tf.keras.layers.Dense(1)])

In [16]:
model.compile(optimizer='Adam',loss='mse',metrics=['mse'])

In [17]:
norm_layer.adapt(X_train)

In [18]:
history = model.fit(X_train,target,epochs=50)

Epoch 1/50
5/5 [==============================] - 1s 7ms/step - loss: 0.2129 - mse: 0.2129
Epoch 2/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0736 - mse: 0.0736
Epoch 3/50
5/5 [==============================] - 0s 5ms/step - loss: 0.0494 - mse: 0.0494
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0391 - mse: 0.0391
Epoch 5/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0298 - mse: 0.0298
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0295 - mse: 0.0295
Epoch 7/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0256 - mse: 0.0256
Epoch 8/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0215 - mse: 0.0215
Epoch 9/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0202 - mse: 0.0202
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0185 - mse: 0.0185
Epoch 11/50
5/5 [==============================] - 0s 5ms/step - loss: 0.0132 - mse: 0.01

In [19]:
test_data.head(1)

,City,Type,P1,P2,P3,P4,P5,P6,P7,P8,...,P30,P31,P32,P33,P34,P35,P36,P37,City_grp,Age
0,0,1,1,4.0,4.0,4.0,1,2,5,4,...,0,0,0,0,0,0,0,0,0,4.499


In [20]:
test_data['Type'].replace(['MB'],[3],inplace=True)

In [21]:
data = model.predict(test_data[:])*1e7
type(test_data)
data = pd.DataFrame(data)

3125/3125 [==============================] - 7s 2ms/step


In [22]:
data.to_csv('Submission2.csv')